In [153]:
%reset -sf

In [154]:
# LOAD THIS BEFORE START, run rmdl

#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

# import numpy as np
import scipy
import matplotlib.pyplot as plt

abc = "abcdefghijklmnopqrstuvwxyz"
abc_map = {c:i for i,c in enumerate(abc)}
M9 = 10**9 + 7  # 998244353
yes, no = "YES", "NO"
d4 = [(1,0),(0,1),(-1,0),(0,-1)]
d8 = [
    (-1,-1),(-1,0),(-1,1),
    (0,-1),(0,0),(0,1),
    (1,-1),(1,0),(1,1)]
d6 = [(2,0),(1,1),(-1,1),(-2,0),(-1,-1),(1,-1)]  # hexagonal layout
MAXINT = sys.maxsize

In [181]:
with open("/Users/htong/Downloads/input.txt") as f:
    srr = f.read().strip().split("\n")

In [156]:
srr = """
#.#####################
#.......#########...###
#######.#########.#.###
###.....#.>.>.###.#.###
###v#####.#v#.###.#.###
###.>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>.#
#.#.#v#######v###.###v#
#...#.>.#...>.>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>.>.#.>.###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################.#
""".strip().split("\n")

In [182]:
arr = [list(row) for row in srr]

In [183]:
n = len(arr)
m = len(arr[0])

In [184]:
class DisjointSet:
    # github.com/not522/ac-library-python/blob/master/atcoder/dsu.py
    # faster implementation of DSU
    def __init__(self, n: int = 0) -> None:
        if n > 0:  # constant size DSU
            self.parent_or_size = [-1] * n
        else:
            # WARNING: non-negative numeric elements only
            self.parent_or_size = defaultdict(lambda: -1)

    def union(self, a: int, b: int) -> int:
        x = self.find(a)
        y = self.find(b)

        if x == y:
            return x

        if -self.parent_or_size[x] < -self.parent_or_size[y]:
            x, y = y, x

        self.parent_or_size[x] += self.parent_or_size[y]
        self.parent_or_size[y] = x

        return x

    def find(self, a: int) -> int:
        parent = self.parent_or_size[a]
        while parent >= 0:
            if self.parent_or_size[parent] < 0:
                return parent
            self.parent_or_size[a], a, parent = (
                self.parent_or_size[parent],
                self.parent_or_size[parent],
                self.parent_or_size[self.parent_or_size[parent]],
            )
        return a

    def size(self, a: int) -> int:
        return -self.parent_or_size[self.find(a)]

In [185]:
# assume DAG

In [186]:
ds = DisjointSet(n*m)

In [187]:
def parse(x,y):
    return x*m + y

def divide(z):
    return divmod(z, m)

In [188]:
for x in range(n):
    for y in range(m):
        if arr[x][y] == "#":
            continue
        for dx, dy in d4:
            xx = x+dx
            yy = y+dy
            if not (0 <= xx < n and 0 <= yy < m):
                continue
            if arr[x][y] == "." and arr[xx][yy] == ".":
                ds.union(parse(x,y), parse(xx,yy))              

In [189]:
mapping = defaultdict(list)

In [190]:
for x in range(n):
    for y in range(m):
        if arr[x][y] == ">":
            mapping[ds.find(parse(x,y-1))].append((ds.find(parse(x,y+1)), 1 + ds.size(parse(x,y+1))))
        if arr[x][y] == "<":
            mapping[ds.find(parse(x,y+1))].append((ds.find(parse(x,y-1)), 1 + ds.size(parse(x,y-1))))
        if arr[x][y] == "^":
            mapping[ds.find(parse(x+1,y))].append((ds.find(parse(x-1,y)), 1 + ds.size(parse(x-1,y))))
        if arr[x][y] == "v":
            mapping[ds.find(parse(x-1,y))].append((ds.find(parse(x+1,y)), 1 + ds.size(parse(x+1,y))))

In [191]:
def dijkstra(list_of_indexes_and_costs, start):
    # shortest path with nonnegative edge costs
    # leetcode.com/problems/path-with-maximum-probability/
    # leetcode.com/problems/network-delay-time/
    length = len(list_of_indexes_and_costs)
    visited = [False] * length
    weights = [MAXINT] * length
    path = [None] * length
    queue = []
    weights[start] = 0
    heapq.heappush(queue, (0, start))
    while queue:
        g, u = heapq.heappop(queue)
        if visited[u]:
            continue
        visited[u] = True
        for v, w in list_of_indexes_and_costs[u]:
            if not visited[v]:
                f = g + w
                if f < weights[v]:
                    weights[v] = f
                    path[v] = u
                    heapq.heappush(queue, (f, v))
    return path, weights


def dijkstra_with_preprocessing(map_from_node_to_nodes_and_costs, source, target, idxs=set()):
    # this operation is costly, recommend to parse to list_of_indexes_and_costs directly
    # leetcode.com/problems/path-with-maximum-probability/
    # leetcode.com/problems/network-delay-time/
    d = map_from_node_to_nodes_and_costs

    if target not in d:  # destination may not have outgoing paths
        d[target] = []
    if source not in d:
        return MAXINT

    # assign indexes
    if idxs:
        idxs = {k: i for i, k in enumerate(idxs)}
    else:
        idxs = {k: i for i, k in enumerate(d.keys())}

    # populate list of indexes and costs
    list_of_indexes_and_costs = [[] for _ in range(len(idxs))]
    for e, vrr in d.items():
        for v, cost in vrr:
            list_of_indexes_and_costs[idxs[e]].append((idxs[v], cost))

    _, costs = dijkstra(list_of_indexes_and_costs, idxs[source])
    return costs[idxs[target]]

In [192]:
def topological_sort(map_from_node_to_nodes, all_nodes=set()):
    # leetcode.com/problems/course-schedule-ii/
    indegree_counter = defaultdict(int)
    for lst in map_from_node_to_nodes.values():
        for v in lst:
            indegree_counter[v] += 1

    if not all_nodes:  # assume all nodes are found in the map
        all_nodes = set(indegree_counter.keys()) | set(map_from_node_to_nodes)

    dq = deque([node for node in all_nodes if node not in indegree_counter])

    res = []
    while dq:
        cur = dq.popleft()
        res.append(cur)
        for nex in map_from_node_to_nodes[cur]:
            indegree_counter[nex] -= 1
            if indegree_counter[nex] == 0:
                dq.append(nex)
    return res if len(res) == len(all_nodes) else []

In [193]:
paths_only = defaultdict(list)
for k,v in mapping.items():
    for x,_ in v:
        paths_only[k].append(x)

In [203]:
# mapping

In [204]:
# paths_only

In [205]:
# topo = topological_sort(paths_only)

In [206]:
# topo

In [198]:
costs = defaultdict(int)

In [199]:
for x in topo:
    for nex, cost in mapping[x]:
        costs[nex] = max(costs[nex], costs[x] + cost)

In [202]:
len(mapping)

94

In [200]:
# mapping

In [207]:
costs[ds.find(parse(n-1,m-2))] + ds.size(parse(0,1)) - 1

2250

In [178]:
ds.size(parse(0,1))

14

In [177]:
ds.size(parse(n-1,m-2))

4

In [124]:
dijkstra_with_preprocessing(mapping, ds.find(parse(0,1)), ds.find(parse(n-1,m-2)))

51

In [125]:
ds.find(parse(0,1))

1

In [53]:
mapping

defaultdict(list,
            {80: [(120, 19), (40, 27), (126, 21)],
             40: [(80, 1), (274, 1)],
             118: [(1, 14), (120, 19), (208, 19)],
             126: [(80, 1), (312, 1)],
             120: [(118, 1), (80, 1)],
             208: [(118, 1), (304, 1)],
             274: [(40, 27), (270, 15), (364, 7)],
             304: [(208, 19), (308, 9), (346, 35)],
             312: [(126, 21), (308, 9), (270, 15), (356, 7)],
             364: [(274, 1), (456, 1)],
             308: [(304, 1), (312, 1)],
             270: [(312, 1), (274, 1)],
             346: [(304, 1), (450, 1)],
             356: [(312, 1), (450, 1)],
             450: [(356, 7), (346, 35), (452, 7)],
             456: [(364, 7), (452, 7), (502, 4)],
             452: [(450, 1), (456, 1)],
             502: [(456, 1)],
             527: [],
             1: [(118, 1)]})